In [2]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import random

import tensorflow as tf
from sklearn import linear_model

import tensorflow.python.platform

import numpy as np
import tensorflow as tf


import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
dir_path = os.path.join(os.getcwd(), 'tok_vnexpress')
categories = list()

data = list()

for directory in os.listdir(dir_path):
    i = 0
    if '.' not in directory:
        list_file_path = os.path.join(dir_path, directory)
        count = 0
        for file_name in os.listdir(list_file_path):
            data_dict = dict()
            data_dict['category'] = directory
            file_path = os.path.join(list_file_path, file_name)
            file = open(file_path,'r')
            data_dict['data'] = file.read()
            data.append(data_dict)
            i = i + 1
            if i == 100: ## For test, only get 100 document from each category
                break

data_df = pd.DataFrame(data)

In [4]:
contents = data_df['data']

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5)
data_vector = vectorizer.fit_transform(contents)

data_contents = data_vector.toarray()

In [5]:
def convert_category_to_number(category):
    if category == "the-thao":
        return 0
    if category == "du-lich":
        return 1
    if category == "khoa-hoc":
        return 2
    if category == "kinh-doanh":
        return 3
    if category == "giai-tri":
        return 4
    if category == "oto-xe-may":
        return 5
    if category == "thoi-su":
        return 6
    if category == "giao-duc":
        return 7
    if category == "the-gioi":
        return 8
    if category == "phap-luat":
        return 9
    if category == "so-hoa":
        return 10
    
    raise ValueError('Category ' + category + "not in training set")

    

In [6]:
df_vectors = pd.DataFrame(data_contents)
df_vectors['category'] = data_df['category'].apply(convert_category_to_number)
df_vectors.to_csv("docs_to_vec.csv")

In [7]:
import tensorflow.python.platform

import numpy as np
import tensorflow as tf


import pandas as pd

# Global variables.
NUM_LABELS = 11    # The number of labels.
BATCH_SIZE = 100  # The number of training examples to use per training step.


# Extract numpy representations of the labels and features given rows consisting of:
#   label, feat_0, feat_1, ..., feat_n

def extract_data(vector_df):

    # Arrays to hold the labels and feature vectors.
    labels = vector_df['category']
    fvecs = vector_df.drop('category', 1)

    # Convert the array of float arrays into a numpy float matrix.
    fvecs_np = np.matrix(fvecs).astype(np.float32)

    # Convert the array of int labels into a numpy array.
    labels_np = np.array(labels).astype(dtype=np.uint8)
    
    # Convert the int numpy array into a one-hot matrix.
    labels_onehot = (np.arange(NUM_LABELS) == labels_np[:, None]).astype(np.float32)
    
    # Return a pair of the feature matrix and the one-hot label matrix.

    return fvecs_np,labels_onehot

In [8]:
def run():
    train_df_vector, test_df_vector = train_test_split(df_vectors, test_size = 0.3)
    
    # Extract it into numpy matrices.
    train_data,train_labels = extract_data(train_df_vector)
    test_data, test_labels = extract_data(test_df_vector)

    # Get the shape of the training data.
    train_size,num_features = train_data.shape


    # This is where training samples and labels are fed to the graph.
    # These placeholder nodes will be fed a batch of training data at each
    # training step using the {feed_dict} argument to the Run() call below.
    x = tf.placeholder("float", shape=[None, num_features])
    y_ = tf.placeholder("float", shape=[None, NUM_LABELS])
    
    # These are the weights that inform how much each feature contributes to
    # the classification.
    W = tf.Variable(tf.zeros([num_features,NUM_LABELS]))
    b = tf.Variable(tf.zeros([NUM_LABELS]))
    y = tf.nn.softmax(tf.matmul(x,W) + b)

    # Optimization.
    cross_entropy = -tf.reduce_sum(y_*tf.log(y))
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)


    # For the test data, hold the entire dataset in one constant node.
    test_data_node = tf.constant(test_data)

    # Evaluation.
    predicted_class = tf.argmax(y,1);
    
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    # Create a local session to run this computation.
    with tf.Session() as s:

        # Run all the initializers to prepare the trainable parameters.
        tf.initialize_all_variables().run()
        
        # Iterate and train.
        print("train_size: ", train_size)
        print("batch size: ", BATCH_SIZE)
        print("train_size // BATCH_SIZE: ", train_size // BATCH_SIZE)
        for step in range(train_size // BATCH_SIZE + 1 )
            offset = (step * BATCH_SIZE) % train_size

            # get a batch of data
            batch_data = train_data[offset:(offset + BATCH_SIZE), :]
            batch_labels = train_labels[offset:(offset + BATCH_SIZE)]

            # feed data into the model
            train_step.run(feed_dict={x: batch_data, y_: batch_labels})
            
            
        print ("Accuracy:", accuracy.eval(feed_dict={x: test_data, y_: test_labels}))
        eval_fun = lambda X: predicted_class.eval(feed_dict={x:X}); 
        
        predict_classes = predicted_class.eval(feed_dict={x:test_data})
        actual_classes = test_labels.argmax(axis=1)
        return predict_classes, actual_classes

SyntaxError: invalid syntax (<ipython-input-8-59c35e6fae0f>, line 48)

In [ ]:
predict_classes, actual_classes = run()
pd.crosstab(actual_classes, predict_classes, margins=True)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(actual_classes, predict_classes, average='macro')